<a href="https://colab.research.google.com/github/vitaliy-sharandin/data_science_projects/blob/master/portfolio/eda/AI_risks_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasets

* AI Incident Database <br>
https://www.kaggle.com/datasets/konradb/ai-incident-database
* Government AI Readiness Index <br>
https://www.statista.com/statistics/1231685/worldwide-government-artificial-intelligence-readiness-index/ <br>
https://www.statista.com/statistics/1231719/eastern-europe-government-artificial-intelligence-readiness-index/
* AI Ethics Guidelines Global Inventory <br>
https://www.statista.com/statistics/1286900/ai-ethics-principles-by-organization-type/


# Questions

1. What are main AI failure types?
2. Failure types evolution through time.
3. Failure types by domain/company/country/demographics/models.
4. Consequences of AI failures
5. How these AI risks map onto existing ethical guidelines and regulations?
6. What are the mitigation strategies that are most effective?
7. How to prevent those risks?



In [ ]:
!pip install -U -q datasets
!pip install -U -q ydata-profiling
!pip install -U -q keybert
!pip install -U -q keyphrase-vectorizers
!pip install -U -q spacy
!python -m spacy download en_core_web_md

In [ ]:
from datasets import load_dataset
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer
from sentence_transformers import SentenceTransformer, util
import spacy

In [ ]:
incident_dataset = load_dataset("vitaliy-sharandin/ai-incidents")
incident_dataset = incident_dataset['train'].to_pandas()

In [ ]:
profile = ProfileReport(incident_dataset, title="Fraud data report", dark_mode=True)
profile.to_notebook_iframe()

# Distribution of incidents over time

## Amount through time

In [ ]:
incident_dataset['date'] = pd.to_datetime(incident_dataset['date'])
incident_dataset['year'] = incident_dataset['date'].dt.year
plt.figure(figsize=(12, 6))
sns.histplot(incident_dataset['year'], bins=40, kde=True)
plt.title('Distribution of AI Incidents Over Time')
plt.xlabel('Year')
plt.ylabel('Number of Incidents')
plt.grid(True)
plt.show()

## Insights
* Early Years: There are very few incidents reported before the year 2010, indicating either a lack of reporting mechanisms or fewer AI deployments during that period.
* Rapid Increase: There is a noticeable increase in the number of incidents starting from around 2015. This could correlate with the broader adoption of AI technologies in various industries.
* Recent Years: The number of incidents seems to peak around 2020 and then shows a slight decline. This could be due to improved AI safety measures, changes in reporting, or other factors that need further investigation.

# Deployers, developers

In [ ]:
def count_occurrences(column):
    items = column.str.replace('[\[\]"]', '', regex=True).str.split(',')
    counter = Counter([item.strip() for sublist in items.dropna() for item in sublist])
    return counter

deployer_counts = count_occurrences(incident_dataset['Alleged deployer of AI system'])
developer_counts = count_occurrences(incident_dataset['Alleged developer of AI system'])


deployer_df = pd.DataFrame(deployer_counts.items(), columns=['Deployer', 'Count']).sort_values('Count', ascending=False).head(30)
developer_df = pd.DataFrame(developer_counts.items(), columns=['Developer', 'Count']).sort_values('Count', ascending=False).head(30)


fig, axes = plt.subplots(1, 2, figsize=(20, 7))

sns.barplot(x='Count', y='Deployer', data=deployer_df, ax=axes[0])
axes[0].set_title('Top 10 Deployers Involved in AI Incidents')
axes[0].set_xlabel('Number of Incidents')

sns.barplot(x='Count', y='Developer', data=developer_df, ax=axes[1])
axes[1].set_title('Top 10 Developers Involved in AI Incidents')
axes[1].set_xlabel('Number of Incidents')

plt.tight_layout()
plt.show()

In [ ]:
## Use Kmeans clustering to cluster into groups

## Insights

  * Most Frequent Deployers: The company "Facebook" appears to be the most frequent deployer involved in AI incidents, followed by companies like "Google" and "Microsoft". This could indicate that platforms with large user bases and extensive AI deployments are more prone to incidents.
  * Most Frequent Developers: Similarly, "Facebook" and "Google" are among the top developers involved in AI incidents. This isn't surprising given their role as major technology companies with extensive AI research and deployment.
  * Tech Giants: Noticeably, many of the top deployers and developers are tech giants, which could imply a higher level of responsibility for these organizations in ensuring AI safety and ethics.

# Harmed parties analysis

In [ ]:
harmed_counts = count_occurrences(incident_dataset['Alleged harmed or nearly harmed parties'])

harmed_df = pd.DataFrame(harmed_counts.items(), columns=['Harmed Party', 'Count']).sort_values('Count', ascending=False).head(20)

plt.figure(figsize=(20, 7))
sns.barplot(x='Count', y='Harmed Party', data=harmed_df)
plt.title('Top 10 Most Commonly Harmed Parties in AI Incidents')
plt.xlabel('Number of Incidents')

plt.show()

In [ ]:
## Use Kmeans clustering to cluster into groups

## Insights


  * General Users: The category "facebook-users" tops the list, indicating that general users of platforms like Facebook are most frequently impacted by AI incidents. This could reflect the broad user base and extensive AI systems deployed by such platforms.

  * Children and Women: Both "children" and "women" appear in the list, suggesting that certain demographic groups may be disproportionately affected by AI incidents.

  * Multiple Categories: We also see categories like "pedestrians" and "drivers," indicating that AI systems deployed in transportation and public spaces have also led to incidents affecting these groups.

  * Public and Private Entities: Various entities like "employees," "law-enforcement," and "companies" also make it to the list, indicating that the impact of AI incidents isn't limited to individual users but also extends to organizations and institutions.

# AI failure types

In [ ]:
kw_model = KeyBERT()
kph_vectorizer = KeyphraseCountVectorizer()

kw_model = KeyBERT()
st_model = SentenceTransformer('distilbert-base-nli-mean-tokens')

def pure_keybert(text, filter_phrase="inappropriate and failed"):
    keywords_tuples = kw_model.extract_keywords(text, vectorizer=kph_vectorizer)
    keywords = [kw[0] for kw in keywords_tuples]

    # Filter out single-word keywords, retain only multi-word keywords
    multi_word_keywords = [kw for kw in keywords if ' ' in kw]

    # If there are multi-word keywords, use them; otherwise, revert to single-word keywords
    keywords_to_use = multi_word_keywords if multi_word_keywords else keywords

    # keyword_embeddings = st_model.encode(keywords_to_use, convert_to_tensor=True)
    # incident_reason_embedding = st_model.encode(filter_phrase, convert_to_tensor=True)

    # similarities = [util.pytorch_cos_sim(keyword_embedding, incident_reason_embedding).item() for keyword_embedding in keyword_embeddings]

    # max_similarity_index = similarities.index(max(similarities))
    # most_relevant_keyword = keywords[max_similarity_index]

    return keywords_to_use[0]

def keybert_with_filter(text, filter_phrase="negative cause"):
    keywords_tuples = kw_model.extract_keywords(text, vectorizer=kph_vectorizer)
    keywords = [kw[0] for kw in keywords_tuples]

    keyword_embeddings = st_model.encode(keywords, convert_to_tensor=True)
    incident_reason_embedding = st_model.encode(filter_phrase, convert_to_tensor=True)

    similarities = [util.pytorch_cos_sim(keyword_embedding, incident_reason_embedding).item() for keyword_embedding in keyword_embeddings]

    max_similarity_index = similarities.index(max(similarities))
    most_relevant_keyword = keywords[max_similarity_index]

    return most_relevant_keyword if similarities[max_similarity_index] > 0.5 else keywords[0]


nlp = spacy.load('en_core_web_md')

def spacy_keyword(text, incident_verbs_list=["fail", "crash", "stop", "halt", "break", "malfunction", "kill", "die", "hurt"], similarity_threshold=0.5):
    doc = nlp(text)
    causes = []
    similar_verbs = []

    # Identify similar or same words to those in the incident_verbs_list
    for token in doc:
        for incident_verb in incident_verbs_list:
            if token.similarity(nlp(incident_verb)) > similarity_threshold:
                similar_verbs.append(token)
                break

    # Extract related noun chunks for identified similar verbs
    for token in similar_verbs:
        for chunk in doc.noun_chunks:
            if token in list(chunk.root.children) or token == chunk.root:
                causes.append(chunk.text)

    return causes

incident_dataset['pure_keybert'] = incident_dataset['description'].head(30).apply(pure_keybert)
incident_dataset['keybert_with_filter'] = incident_dataset['description'].head(30).apply(keybert_with_filter)
incident_dataset['spacy'] = incident_dataset['description'].head(30).apply(spacy_keyword)
incident_dataset[['description', 'pure_keybert','keybert_with_filter', 'spacy']]

In [ ]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")

def extract_impacted_entities(text):
    doc = nlp(text)

    # Using NER to get potential entities
    entities = [ent.text for ent in doc.ents if ent.label_ in ["ORG", "PERSON"]]

    # Using dependency parsing to refine the entities list
    matcher = Matcher(nlp.vocab)

    # Define a pattern to match verbs like 'affect', 'impact', etc. and their objects
    pattern = [{"LEMMA": {"IN": ["affect", "impact", "hurt"]}},
               {"POS": "ADP", "OP": "?"},  # optional preposition
               {"POS": "NOUN", "OP": "?"}, # optional noun
               {"POS": "DET", "OP": "?"},  # optional determiner
               {"POS": {"IN": ["NOUN", "PROPN"]}, "OP": "+"}]  # nouns or proper nouns

    matcher.add("IMPACTED_ENTITY_PATTERN", [pattern])
    matches = matcher(doc)

    # Extracting matched entities
    for match_id, start, end in matches:
        span = doc[start:end]
        entities.append(span.text)

    # Return unique entities
    return list(set(entities))



1. Word cloud from keybert/ keyphrases. Is WordCloud used for phrases?
2. Clustering of results into groups

